# Tourists at Home

### Executive Summary

This project aims to understand the local perception of Domestic Tourism and to provide further insights for STB to plan its next phase of the #SingapoRediscovers campaign. By targetting Singaporeans more effectively, we aim to build up a sustainable economy of domestic tourism based on renewed and sustained interest in their own home country, which should serve our tourism industry well both during and after Covid-19.

This notebook imports the DataFrames we have compiled from our Scrape notebook 1.0 for cleaning and subsequent merging so we can perform our EDA in the next notebook. Between cleaning and merging, we also perform features engineering as well as imputation for missing data in this notebook. Notable features engineered here include meta-features such as parsing the datetime column correctly and generating topic tags. For imputation, the strategy is always to deduce where possible and infer otherwise, before considering dropping the incomplete data points if they are only a small subset of our dataset.

#### Content

- [Instagram](#Instagram)
- [YouTube Videos](#YouTube-Videos)
- [YouTube Comments](#YouTube-Comments)

#### Python Libraries

In [1]:
import pandas as pd
import numpy as np

from nltk.corpus import stopwords
import string
import re

### Instagram

#### Import Scrapes

In [2]:
df_insta_rediscoversg = pd.read_csv("../datasets/df_insta_rediscoversg_2020-10-18.csv", parse_dates=["date_time"])
df_insta_rediscoversg.shape

(2352, 18)

In [3]:
# ensure that there are no duplicate scrapes within df
df_insta_rediscoversg["post_id"].nunique()

2352

In [4]:
df_insta_rediscoversingapore = pd.read_csv("../datasets/df_insta_rediscoversingapore_2020-10-19.csv", parse_dates=["date_time"])
df_insta_rediscoversingapore.shape

(1226, 18)

In [5]:
# ensure that there are no duplicate scrapes within df
df_insta_rediscoversingapore["post_id"].nunique()

1226

In [6]:
df_insta_singapoliday = pd.read_csv("../datasets/df_insta_singapoliday_2020-10-19.csv", parse_dates=["date_time"])
df_insta_singapoliday.shape

(1445, 18)

In [7]:
# ensure that there are no duplicate scrapes within df
df_insta_singapoliday["post_id"].nunique()

1445

In [8]:
df_insta_singaporeliday = pd.read_csv("../datasets/df_insta_singaporeliday_2020-10-19.csv", parse_dates=["date_time"])
df_insta_singaporeliday.shape

(400, 18)

In [9]:
# ensure that there are no duplicate scrapes within df
df_insta_singaporeliday["post_id"].nunique()

400

In [10]:
df_insta_madaboutsingapore2020 = pd.read_csv("../datasets/df_insta_madaboutsingapore2020_2020-10-20.csv", parse_dates=["date_time"])
df_insta_madaboutsingapore2020.shape

(4600, 18)

In [11]:
# ensure that there are no duplicate scrapes within df
df_insta_madaboutsingapore2020["post_id"].nunique()

4600

In [12]:
df_insta_madaboutsingapore2020c = pd.read_csv("../datasets/df_insta_madaboutsingapore2020c_2020-10-20.csv", parse_dates=["date_time"])
df_insta_madaboutsingapore2020c.shape

(1250, 18)

In [13]:
# ensure that there are no duplicate scrapes within df
df_insta_madaboutsingapore2020c["post_id"].nunique()

1250

We have multiple CSVs saved out from scraping #SingapoRediscovers with expected duplicates. Since it is our official hashtag, there is greater effort to scrape more current posts and in batches.

In [15]:
df_insta_singaporediscovers_1 = pd.read_csv("../datasets/df_insta_singaporediscovers_2020-10-12.csv", parse_dates=["date_time"])
df_insta_singaporediscovers_1.shape

(3104, 18)

In [16]:
# ensure that there are no duplicate scrapes within df
df_insta_singaporediscovers_1["post_id"].nunique()

3104

In [17]:
df_insta_singaporediscovers_2 = pd.read_csv("../datasets/df_insta_singaporediscovers_2020-10-20.csv", parse_dates=["date_time"])
df_insta_singaporediscovers_2.shape

(1981, 18)

In [18]:
# ensure that there are no duplicate scrapes within df
df_insta_singaporediscovers_2["post_id"].nunique()

1981

In [19]:
df_insta_singaporediscovers_3 = pd.read_csv("../datasets/df_insta_singaporediscovers_2020-10-21.csv", parse_dates=["date_time"])
df_insta_singaporediscovers_3.shape

(1787, 18)

In [20]:
# ensure that there are no duplicate scrapes within df
df_insta_singaporediscovers_3["post_id"].nunique()

1787

In [21]:
df_insta_singaporediscovers_4 = pd.read_csv("../datasets/df_insta_singaporediscovers_2020-10-22.csv", parse_dates=["date_time"])
df_insta_singaporediscovers_4.shape

(4199, 18)

In [22]:
# ensure that there are no duplicate scrapes within df
df_insta_singaporediscovers_4["post_id"].nunique()

4199

#### Merge DataFrames

In [23]:
def compile_unique_insta(df1, df2):
    
    # find duplicate posts that share target hashtags
    post_id_intersects = set(df1["post_id"]).intersection(set(df2["post_id"]))
    print("Duplicate posts removed:", len(post_id_intersects))
    
    # use df1 as master copy to add to
    # drop duplicates from df2
    # concat as single df without duplicates
    dups = df2["post_id"].isin(post_id_intersects)
    df_concat = pd.concat([df1, df2[~dups]])
    
    return df_concat

In [24]:
# remove duplicate posts with shared target hashtags
df_insta = compile_unique_insta(df_insta_rediscoversg, df_insta_rediscoversingapore)
df_insta.shape

Duplicate posts removed: 93


(3485, 18)

In [25]:
# remove duplicate posts with shared target hashtags
df_insta = compile_unique_insta(df_insta, df_insta_singapoliday)
df_insta.shape

Duplicate posts removed: 159


(4771, 18)

In [26]:
# remove duplicate posts with shared target hashtags
df_insta = compile_unique_insta(df_insta, df_insta_singaporeliday)
df_insta.shape

Duplicate posts removed: 5


(5166, 18)

In [27]:
# remove duplicate posts with shared target hashtags
df_insta = compile_unique_insta(df_insta, df_insta_madaboutsingapore2020)
df_insta.shape

Duplicate posts removed: 19


(9747, 18)

In [28]:
# remove duplicate posts with shared target hashtags
df_insta = compile_unique_insta(df_insta, df_insta_madaboutsingapore2020c)
df_insta.shape

Duplicate posts removed: 149


(10848, 18)

In [29]:
# remove duplicate posts with shared target hashtags
df_insta = compile_unique_insta(df_insta, df_insta_singaporediscovers_1)
df_insta.shape

Duplicate posts removed: 414


(13538, 18)

In [30]:
# remove duplicate posts with shared target hashtags
df_insta = compile_unique_insta(df_insta, df_insta_singaporediscovers_2)
df_insta.shape

Duplicate posts removed: 1248


(14271, 18)

In [31]:
# remove duplicate posts with shared target hashtags
df_insta = compile_unique_insta(df_insta, df_insta_singaporediscovers_3)
df_insta.shape

Duplicate posts removed: 1697


(14361, 18)

In [32]:
# remove duplicate posts with shared target hashtags
df_insta = compile_unique_insta(df_insta, df_insta_singaporediscovers_4)
df_insta.shape

Duplicate posts removed: 4024


(14536, 18)

In [33]:
# reset index for final df
df_insta.reset_index(drop=True, inplace=True)

#### Data Overview

In [34]:
df_insta.head(3)

,post_id,post_slug,unix_time,date_time,post_caption,hashtags,topic_tags,is_video,is_ad,post_likes,geo_tag,geo_slug,owner_id,owner_verified,owner_privacy,owner_unpublished,owner_total_posts,owner_total_followers
0,1006641971264095607,34T4oZMgV3,1434221095,2015-06-13 18:44:55,Oh look! Natural heart shaped form of Ivan Hen...,"['#PinkDotSg', '#WhereLovesLiveSg', '#Rediscov...",['2 people.'],False,False,40,Pinkdot @ Hong Lim Park,pinkdot-hong-lim-park,196778200,False,False,False,5205,1232
1,1202880298563729182,BCxfUInNPce,1457614527,2016-03-10 12:55:27,.\n.\n.\n.\n.\n.\n#exploresingapore #instasg #...,"['#exploresingapore', '#instasg', '#gf_singapo...",['indoor.'],False,False,33,Yangtze Cinema,yangtze-cinema,33123388,False,False,False,742,995
2,677102977071319026,lljZH4sgfy,1394936986,2014-03-16 02:29:46,On the road to hell..... Reliving Haw Par Vill...,"['#rediscoversg', '#Singapore']",['Photo by Belinda Tan in 虎豹别墅.'],False,False,3,虎豹别墅,NaN,196778200,False,False,False,5205,1232


In [35]:
# check that there are no duplicate scrapes
df_insta.duplicated(keep="first").value_counts()

False    14536
dtype: int64

In [36]:
df_insta.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 14536 entries, 0 to 14535
Data columns (total 18 columns):
 #   Column                 Non-Null Count  Dtype         
---  ------                 --------------  -----         
 0   post_id                14536 non-null  int64         
 1   post_slug              14536 non-null  object        
 2   unix_time              14536 non-null  int64         
 3   date_time              14536 non-null  datetime64[ns]
 4   post_caption           14485 non-null  object        
 5   hashtags               14486 non-null  object        
 6   topic_tags             10819 non-null  object        
 7   is_video               14536 non-null  bool          
 8   is_ad                  14536 non-null  bool          
 9   post_likes             14536 non-null  int64         
 10  geo_tag                10035 non-null  object        
 11  geo_slug               9966 non-null   object        
 12  owner_id               14536 non-null  int64         
 13  o

In [37]:
# we can observe missing data for "post_caption", "hashtags", "topic_tags", "geo_tag", and "geo_slug"

#### Missing Text

In [38]:
# view posts with missing post captions
df_insta[df_insta["post_caption"].isnull()].shape

(51, 18)

In [39]:
# replace empty fields in hashtags col filled with "[]" from scrape with a true null value
df_insta["hashtags"] = df_insta["hashtags"].replace({"[]": None})

In [40]:
# since instagram was scraped by querying target hashtags
# these target hashtags are likely to skew our model's prediction
# as such, we should drop these target hashtags

target_hash = ["#singaporediscovers", "#rediscoversg", "#rediscoversingapore", "#singapoliday", "#singaporeliday",
               "#madaboutsingapore2020", "#madaboutsingapore2020c"]
target_pattern = "|".join(target_hash)

# drop target hashtags from hashtags col
# we will also drop these target hashtags from the post captions afterwards
df_insta["hashtags"] = df_insta["hashtags"].str.lower().str.replace(target_pattern, "")
df_insta.head(3)

,post_id,post_slug,unix_time,date_time,post_caption,hashtags,topic_tags,is_video,is_ad,post_likes,geo_tag,geo_slug,owner_id,owner_verified,owner_privacy,owner_unpublished,owner_total_posts,owner_total_followers
0,1006641971264095607,34T4oZMgV3,1434221095,2015-06-13 18:44:55,Oh look! Natural heart shaped form of Ivan Hen...,"['#pinkdotsg', '#whereloveslivesg', '#rediscov...",['2 people.'],False,False,40,Pinkdot @ Hong Lim Park,pinkdot-hong-lim-park,196778200,False,False,False,5205,1232
1,1202880298563729182,BCxfUInNPce,1457614527,2016-03-10 12:55:27,.\n.\n.\n.\n.\n.\n#exploresingapore #instasg #...,"['#exploresingapore', '#instasg', '#gf_singapo...",['indoor.'],False,False,33,Yangtze Cinema,yangtze-cinema,33123388,False,False,False,742,995
2,677102977071319026,lljZH4sgfy,1394936986,2014-03-16 02:29:46,On the road to hell..... Reliving Haw Par Vill...,"['', '#singapore']",['Photo by Belinda Tan in 虎豹别墅.'],False,False,3,虎豹别墅,NaN,196778200,False,False,False,5205,1232


In [41]:
# view posts with missing hashtags
df_insta[df_insta["hashtags"].isnull()].shape

(1886, 18)

In [42]:
# find posts with captions but no hashtags
have_capt_no_hash = set(df_insta[df_insta["hashtags"].isnull()].index).difference(set(df_insta[df_insta["post_caption"].isnull()].index))
df_insta.iloc[list(have_capt_no_hash)].shape

(1835, 18)

In [43]:
# 1886 posts w/o hash -  51 posts w/o captions = 1835 posts w/ captions but w/o hash
# this verifies that all posts without captions will not have hashtags
# while all posts without hashtags may still have accompanying captions
### because hashtags were extracted from post captions during scraping
### if there were no post captions, then there were no hashtags too

In [44]:
# view posts with missing topic tags
df_insta[df_insta["topic_tags"].isnull()].head(3)

,post_id,post_slug,unix_time,date_time,post_caption,hashtags,topic_tags,is_video,is_ad,post_likes,geo_tag,geo_slug,owner_id,owner_verified,owner_privacy,owner_unpublished,owner_total_posts,owner_total_followers
6,2406162771994753499,CFkaErwhDnb,1601056976,2020-09-25 18:02:56,Do you know: Sembawang beach is one of the few...,"['#sglgexperience', '#localguides', '#singapor...",NaN,False,False,10,Sembawang Park Fishing Jetty,sembawang-park-fishing-jetty,32045251153,False,False,False,54,48
10,936794654076236572,0AKcZHsvMc,1425894646,2015-03-09 09:50:46,We see the city in a different light. We see t...,"['#parkour', '#parkoursingapore', '#tracer', '...",NaN,True,False,42,NaN,NaN,736533259,False,False,False,1265,1296
21,2397733941879266230,CFGdlD_nFu2,1600052181,2020-09-14 02:56:21,It's so much fun at the @artsciencemuseumsg ! ...,"['#bbleeia', '#3yrs9mthsold', '#sgtoddler', '#...",NaN,False,False,18,NaN,NaN,25853204741,False,False,False,60,61


In [45]:
# find posts without both captions and topic tags - that is to say, no NLP features
no_nlp = set(df_insta[df_insta["topic_tags"].isnull()]["post_id"]).intersection(set(df_insta[df_insta["post_caption"].isnull()]["post_id"]))
no_nlp = df_insta["post_id"].isin(no_nlp)
print("No. of posts without text:", len(df_insta[no_nlp]))
df_insta[no_nlp]

No. of posts without text: 5


,post_id,post_slug,unix_time,date_time,post_caption,hashtags,topic_tags,is_video,is_ad,post_likes,geo_tag,geo_slug,owner_id,owner_verified,owner_privacy,owner_unpublished,owner_total_posts,owner_total_followers
6748,2355897232517229971,CCx1A0Hg-2T,1595064907,2020-07-18 09:35:07,NaN,None,NaN,True,False,48,NaN,NaN,30679523,False,False,False,560,43906
9353,2151641071654035438,B3cKiegFF_u,1570715625,2019-10-10 13:53:45,NaN,NaN,NaN,False,False,109,Singapore,singapore,1494605314,False,False,False,269,717
11762,2401679332442353362,CFUeqDSn67S,1600522508,2020-09-19 13:35:08,NaN,NaN,NaN,False,False,22,Chinatown,chinatown,6654500502,False,False,False,80,74
11854,2396610934708387097,CFCePKEHD0Z,1599918308,2020-09-12 13:45:08,NaN,NaN,NaN,False,False,85,NaN,NaN,19332530737,False,False,False,51,369
12135,2407346391717164090,CFonMnhnrQ6,1601198074,2020-09-27 09:14:34,NaN,NaN,NaN,False,False,23,Chinatown,chinatown,6654500502,False,False,False,80,74


Since this is an NLP project, posts without words cannot offer NLP training for the model. As there are only 5 instagram posts without post captions and topic tags out of over 14,000 posts scraped and therefore no associated word features, this is a rare occurence. We shall just drop these 5 instagram posts.

In [46]:
df_insta.shape # current shape of df

(14536, 18)

In [47]:
df_insta = df_insta.drop(df_insta[no_nlp].index).reset_index(drop=True)
df_insta.shape # verify that 5 rows were dropped

(14531, 18)

In [48]:
# after removing data points with no captions and topic tags
# we are left with only rows with captions and/or topic tags
# remaining data points therefore all have NLP word features that our model can train on
# we should merge all word features into a single column for easy modelling later

#### Text Cleaning

In [49]:
df_insta.head(1)

,post_id,post_slug,unix_time,date_time,post_caption,hashtags,topic_tags,is_video,is_ad,post_likes,geo_tag,geo_slug,owner_id,owner_verified,owner_privacy,owner_unpublished,owner_total_posts,owner_total_followers
0,1006641971264095607,34T4oZMgV3,1434221095,2015-06-13 18:44:55,Oh look! Natural heart shaped form of Ivan Hen...,"['#pinkdotsg', '#whereloveslivesg', '#rediscov...",['2 people.'],False,False,40,Pinkdot @ Hong Lim Park,pinkdot-hong-lim-park,196778200,False,False,False,5205,1232


In [50]:
df_insta.shape

(14531, 18)

In [51]:
# clean text for modelling

def remove_newline(text):
    return text.replace(r"\n", " ")

def remove_url(text):
    url = re.compile(r"http?://\S+|https?://\S+|www\.\S+")
    return url.sub(r"", text)

def remove_html(text):
    html = re.compile(r"<.*?>")
    return html.sub(r"", text)

def remove_emoji(text):
    emoji_pattern = re.compile(
        "["
        u"\U0001F600-\U0001F64F"
        u"\U0001F300-\U0001F5FF"
        u"\U0001F680-\U0001F6FF"
        u"\U0001F1E0-\U0001F1FF"
        u"\U00002702-\U000027B0"
        u"\U000024C2-\U0001F251"
        "]+",
        flags=re.UNICODE
    )
    return emoji_pattern.sub(r"", text)

def remove_country_tags(text):
    country_tags = re.compile(r"[🇦🇧🇨🇩🇪🇫🇬🇭🇮🇯🇰🇱🇲🇳🇴🇵🇶🇷🇸🇹🇺🇻🇼🇽🇾🇿]") # country tags uses non-ascii characters
    return country_tags.sub(r"", text)

def remove_punct(text):
    table = str.maketrans("", "", string.punctuation.replace("#", "").replace("'", "")) # add exceptions through replace
    return text.translate(table)

def remove_stopwords(text):
    stop = set(stopwords.words("english")) # use list of stop words from nltk
    # list of more stop words to remove discovered when modelling
    more_stop = ["love", "like", "singapore", "sg"]
    for s in more_stop:
        stop.add(s)
    text = [word.lower() for word in text.split() if word.lower() not in stop]
    return " ".join(text)

def clean_text(df, target):
    cleaned = df[target].map(lambda x: remove_newline(str(x).lower()))
    cleaned = cleaned.map(lambda x: remove_url(str(x).lower()))
    cleaned = cleaned.map(lambda x: remove_html(str(x).lower()))
    cleaned = cleaned.map(lambda x: remove_emoji(str(x).lower()))
    cleaned = cleaned.map(lambda x: remove_country_tags(str(x).lower()))
    cleaned = cleaned.map(lambda x: remove_punct(str(x).lower()))
    cleaned = cleaned.map(lambda x: remove_stopwords(str(x).lower()))
    return cleaned

In [52]:
df_insta["post_caption_clean"] = clean_text(df_insta, "post_caption")
df_insta.shape #verify col added

(14531, 19)

In [53]:
# because scraping for insta posts were done with target hashtags
# we should drop these target hashtags as they are bound to occur overwhelmingly and may skew our modelling

# using target_pattern query from before when dropping target hashtags from hashtags col
df_insta["post_caption_drop_target"] = df_insta["post_caption_clean"].str.lower().str.replace(target_pattern, "")
df_insta.head(3)

,post_id,post_slug,unix_time,date_time,post_caption,hashtags,topic_tags,is_video,is_ad,post_likes,geo_tag,geo_slug,owner_id,owner_verified,owner_privacy,owner_unpublished,owner_total_posts,owner_total_followers,post_caption_clean,post_caption_drop_target
0,1006641971264095607,34T4oZMgV3,1434221095,2015-06-13 18:44:55,Oh look! Natural heart shaped form of Ivan Hen...,"['#pinkdotsg', '#whereloveslivesg', '#rediscov...",['2 people.'],False,False,40,Pinkdot @ Hong Lim Park,pinkdot-hong-lim-park,196778200,False,False,False,5205,1232,oh look natural heart shaped form ivan heng gl...,oh look natural heart shaped form ivan heng gl...
1,1202880298563729182,BCxfUInNPce,1457614527,2016-03-10 12:55:27,.\n.\n.\n.\n.\n.\n#exploresingapore #instasg #...,"['#exploresingapore', '#instasg', '#gf_singapo...",['indoor.'],False,False,33,Yangtze Cinema,yangtze-cinema,33123388,False,False,False,742,995,#exploresingapore #instasg #gfsingapore #gameo...,#exploresingapore #instasg #gfsingapore #gameo...
2,677102977071319026,lljZH4sgfy,1394936986,2014-03-16 02:29:46,On the road to hell..... Reliving Haw Par Vill...,"['', '#singapore']",['Photo by Belinda Tan in 虎豹别墅.'],False,False,3,虎豹别墅,NaN,196778200,False,False,False,5205,1232,road hell reliving haw par villa 15 16 22 23 m...,road hell reliving haw par villa 15 16 22 23 m...


In [54]:
# in case hashtags are overwhelming and generic, let's drop all hashtags too
df_insta["post_caption_drop_hash"] = df_insta["post_caption_clean"].str.extractall(r"(?<!#)(\b[\w']+)").groupby(level=0)[0].apply(lambda x: " ".join(x))
df_insta.head(3)

,post_id,post_slug,unix_time,date_time,post_caption,hashtags,topic_tags,is_video,is_ad,post_likes,...,geo_slug,owner_id,owner_verified,owner_privacy,owner_unpublished,owner_total_posts,owner_total_followers,post_caption_clean,post_caption_drop_target,post_caption_drop_hash
0,1006641971264095607,34T4oZMgV3,1434221095,2015-06-13 18:44:55,Oh look! Natural heart shaped form of Ivan Hen...,"['#pinkdotsg', '#whereloveslivesg', '#rediscov...",['2 people.'],False,False,40,...,pinkdot-hong-lim-park,196778200,False,False,False,5205,1232,oh look natural heart shaped form ivan heng gl...,oh look natural heart shaped form ivan heng gl...,oh look natural heart shaped form ivan heng gl...
1,1202880298563729182,BCxfUInNPce,1457614527,2016-03-10 12:55:27,.\n.\n.\n.\n.\n.\n#exploresingapore #instasg #...,"['#exploresingapore', '#instasg', '#gf_singapo...",['indoor.'],False,False,33,...,yangtze-cinema,33123388,False,False,False,742,995,#exploresingapore #instasg #gfsingapore #gameo...,#exploresingapore #instasg #gfsingapore #gameo...,NaN
2,677102977071319026,lljZH4sgfy,1394936986,2014-03-16 02:29:46,On the road to hell..... Reliving Haw Par Vill...,"['', '#singapore']",['Photo by Belinda Tan in 虎豹别墅.'],False,False,3,...,NaN,196778200,False,False,False,5205,1232,road hell reliving haw par villa 15 16 22 23 m...,road hell reliving haw par villa 15 16 22 23 m...,road hell reliving haw par villa 15 16 22 23 m...


In [55]:
# take a closer look at topic tags
df_insta["topic_tags"][:35].values

array(["['2 people.']", "['indoor.']",
       "['Photo by Belinda Tan in 虎豹别墅.']",
       "['Photo by pixbykm on May 03', '2011.']",
       "['Photo by Belinda Tan in Esplanade – Theatres on the Bay.']",
       "['1 person.']", nan, "['Photo by Jaclynn Seah in 虎豹别墅.']",
       "['1 person.']",
       "['Photo by Belinda Tan in Marina Bay Waterfront Promenade.']",
       nan, "['1 person.']", "['Photo by Phy 🌈 on April 23', '2011.']",
       "['Photo by vanzephyr on May 11', '2011.']",
       "['Photo by Belinda Tan in The Substation.']",
       "['Photo by Keith Teoh on June 02', '2011.']",
       "['Photo by Belinda Tan in 养记鱼头 Smith Street Hawker Center Chinatown.']",
       "['kitchen', 'indoor', 'food.']",
       "['Photo by Belinda Tan in The Coliseum@Hard Rock Hotel', 'Resorts World', 'Sentosa.']",
       "['sky', 'cloud', 'outdoor', 'water', 'nature.']",
       "['Photo by Singapore Street Art in Jalan Klapa.']", nan,
       "['cloud', 'sky', 'bicycle', 'outdoor', 'water.']", na

Studying the topic tags in comparison to other features revealed a few patterns.
- Topic tags may describe the number of people present in the instagram post. For example, `1 person` or `2 people` etc. These are generally not useful to understanding the content of the post.
- Topic tags also aggregated information about the post, such as the medium, author, location, and date. These are captured and aggregated as a string in the format: `{Medium} by {Author} on {Date} in {Location}.`, or i.e. `Photo by Ah Huat on Oct 10, 2020 in Clarke Quay.`. They are repeat information already recorded elsewhere in their respective columns in the DataFrame and are not useful.
- Topic tags may also describe the themes present in the post, such as `'sky', 'cloud', 'outdoor', 'water', 'nature.'`. These tags are useful as they categorises the posts for us.
- Topic tags may also include instagram tags to other instagram accounts, such as `@best.friend` and are prefixed by the "@" character. These tags are useful as they indicate the association desired/expressed by the author.

In [56]:
# extract @tags
# convert returned multi-index into space-separated words
# assign to new col
df_insta["at_tags"] = df_insta["topic_tags"].str.lower().str.extractall("(\@\w+.?\w+)").groupby(level=0)[0].apply(lambda x: " ".join(x))
df_insta.shape # verify new col added

(14531, 22)

In [57]:
# find all topic tags that include the 4 words that indicate posts we do not want
# returned boolean masks include missing values because of missing topic tag values
not_useful = df_insta["topic_tags"].str.contains("(Photo)|(Video)|(person)|(people)")

# missing topic tags are also not useful, impute missing boolean mask values as such
not_useful.fillna(True, inplace=True)
df_insta[~not_useful]["topic_tags"] # view topic tags that are useful

C:\Users\yewto\anaconda3\lib\site-packages\pandas\core\strings.py:1954: UserWarning: This pattern has match groups. To actually get the groups, use str.extract.
  return func(self, *args, **kwargs)


1                                              ['indoor.']
17                          ['kitchen', 'indoor', 'food.']
19         ['sky', 'cloud', 'outdoor', 'water', 'nature.']
22        ['cloud', 'sky', 'bicycle', 'outdoor', 'water.']
30           ['plant', 'tree', 'sky', 'outdoor', 'water.']
                               ...                        
14520                                            ['food.']
14521                                ['night', 'outdoor.']
14526          ["text that says 'BIRTHDAY GIVEAWAY ctm'."]
14527                                          ['indoor.']
14530    ['sky', 'cloud', 'tree', 'plant', 'bridge', 'o...
Name: topic_tags, Length: 5349, dtype: object

In [58]:
# save out space-separated useful tags
df_insta["useful_tags"] = df_insta.loc[~not_useful, "topic_tags"].str.lower().str.extractall(r"(?<!@)(\b\w+)").groupby(level=0)[0].apply(lambda x: " ".join(x))
df_insta.shape # verify new col added

(14531, 23)

In [59]:
# join string if there are both @tags and useful tags
mask = df_insta["at_tags"].notnull() & df_insta["useful_tags"].notnull()
df_insta.loc[mask, "tags"] = df_insta.loc[mask, "useful_tags"] + " " + df_insta.loc[mask, "at_tags"]

# add useful tags if they are the only tags present
mask = df_insta["at_tags"].isnull() & df_insta["useful_tags"].notnull()
df_insta.loc[mask, "tags"] = df_insta.loc[mask, "useful_tags"]

# add at tags if they are the only tags present
mask = df_insta["at_tags"].notnull() & df_insta["useful_tags"].isnull()
df_insta.loc[mask, "tags"] = df_insta.loc[mask, "at_tags"]

df_insta.shape # verify new col added

(14531, 24)

In [60]:
df_insta.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 14531 entries, 0 to 14530
Data columns (total 24 columns):
 #   Column                    Non-Null Count  Dtype         
---  ------                    --------------  -----         
 0   post_id                   14531 non-null  int64         
 1   post_slug                 14531 non-null  object        
 2   unix_time                 14531 non-null  int64         
 3   date_time                 14531 non-null  datetime64[ns]
 4   post_caption              14485 non-null  object        
 5   hashtags                  12650 non-null  object        
 6   topic_tags                10819 non-null  object        
 7   is_video                  14531 non-null  bool          
 8   is_ad                     14531 non-null  bool          
 9   post_likes                14531 non-null  int64         
 10  geo_tag                   10032 non-null  object        
 11  geo_slug                  9963 non-null   object        
 12  owner_id          

In [61]:
# after cleaning up topic tags and post captions, there are now less text than before
# are there now more posts without NLP features?

no_nlp = set(df_insta[df_insta["tags"].isnull()]["post_id"]).intersection(set(df_insta[df_insta["post_caption_drop_target"].isnull()]["post_id"]))
no_nlp = df_insta["post_id"].isin(no_nlp)
print("No. of posts without text:", len(df_insta[no_nlp]))
df_insta[no_nlp]

No. of posts without text: 0


,post_id,post_slug,unix_time,date_time,post_caption,hashtags,topic_tags,is_video,is_ad,post_likes,...,owner_privacy,owner_unpublished,owner_total_posts,owner_total_followers,post_caption_clean,post_caption_drop_target,post_caption_drop_hash,at_tags,useful_tags,tags


In [62]:
# there are only 43 posts without text, which is a small subset of our data
# let's drop these posts
df_insta.drop(df_insta[no_nlp].index, inplace=True)
df_insta.shape

(14531, 24)

#### Merge NLP Features

In [63]:
# merge all NLP features for each post into a single col

In [64]:
# join string if there are both tags and captions
mask = df_insta["post_caption_clean"].notnull() & df_insta["tags"].notnull()
df_insta.loc[mask, "content"] = df_insta.loc[mask, "post_caption_clean"] + " " + df_insta.loc[mask, "tags"]

# add tags if they are the only NLP feature present
mask = df_insta["post_caption_clean"].isnull() & df_insta["tags"].notnull()
df_insta.loc[mask, "content"] = df_insta.loc[mask, "tags"]

# add captions if they are the only NLP feature present
mask = df_insta["post_caption_clean"].notnull() & df_insta["tags"].isnull()
df_insta.loc[mask, "content"] = df_insta.loc[mask, "post_caption_clean"]

df_insta.shape # verify new col added

(14531, 25)

In [65]:
df_insta["content"].isnull().sum() # verify that there are all remaining posts come with text content

0

#### Missing Locations

In [66]:
df_insta["geo_tag"].isnull().sum()

4499

In [67]:
df_insta["geo_slug"].isnull().sum()

4568

In [68]:
# geo tags provide explicit information about the location and are therefore more important
# are there any missing geo tag data with a present geo slug for us to infer and impute from?
set(df_insta[df_insta["geo_tag"].isnull()]["post_id"]).difference(set(df_insta[df_insta["geo_slug"].isnull()]["post_id"]))

set()

In [69]:
# empty set returned, therefore all posts with missing geo tags also have missing geo slugs
# imputation is not possible by inferring from geo slugs

In [70]:
df_insta[df_insta["geo_tag"].isnull()][["post_caption", "topic_tags", "geo_tag"]]

,post_caption,topic_tags,geo_tag
3,Coffeeshop talk....,"['Photo by pixbykm on May 03', '2011.']",NaN
10,We see the city in a different light. We see t...,NaN,NaN
12,#singapore cityscape. I was lucky to capture t...,"['Photo by Phy 🌈 on April 23', '2011.']",NaN
13,人去楼空 Everybody is gone.,"['Photo by vanzephyr on May 11', '2011.']",NaN
15,#window#nightlights#bokeh,"['Photo by Keith Teoh on June 02', '2011.']",NaN
...,...,...,...
14517,Who knew a mere thrift shop could be a cool pl...,"['one or more people', 'people st', 'ing', 'we...",NaN
14520,Bursting ondeh ondeh. Making the coconut was h...,['food.'],NaN
14524,On the Little India Heritage Trail with our wo...,"['2 people', 'outdoor', ""text that says 'LITTL...",NaN
14526,[GIVEAWAY] It's our 47th birthday next Tuesday...,"[""text that says 'BIRTHDAY GIVEAWAY ctm'.""]",NaN


#### Select Instagram Posts

Since we hypothesise that Instagram posts can reveal to us the local public's explicit understanding of domestic tourism, we should therefore be selective of the Instagram posts that we train our model on. For example, we should cut out noise from paid advertisement posts on Instagram or posts from corporations' Instagram account. Advertisements serve only to sell us a product/image/service and may sometimes miss the mark about what we want.

In [72]:
df_insta.columns

Index(['post_id', 'post_slug', 'unix_time', 'date_time', 'post_caption',
       'hashtags', 'topic_tags', 'is_video', 'is_ad', 'post_likes', 'geo_tag',
       'geo_slug', 'owner_id', 'owner_verified', 'owner_privacy',
       'owner_unpublished', 'owner_total_posts', 'owner_total_followers',
       'post_caption_clean', 'post_caption_drop_target',
       'post_caption_drop_hash', 'at_tags', 'useful_tags', 'tags', 'content'],
      dtype='object')

In [73]:
df_insta["is_ad"].value_counts() # find posts that are paid-for advertising

False    14531
Name: is_ad, dtype: int64

In [74]:
# since it is highly unlikely that none of the posts scraped were advertisements borrowing the target hashtags
# the "is_ad" data scraped from instagram appears to be defunct/not updated

In [75]:
df_insta["owner_verified"].value_counts() # find posts whose owner are verified

False    14256
True       275
Name: owner_verified, dtype: int64

In [76]:
# glimpse of verified posts
df_insta[df_insta["owner_verified"]][["content", "hashtags"]]

,content,hashtags
171,checked new apple store town first floating ap...,"['#jetstarasia', '#singapore', '#visitsingapor..."
554,new permanent galleries #nationalmuseumofsinga...,"['#nationalmuseumofsingapore', '#repost', '#na..."
816,plans weekend treat staycay deserve rediscover...,"['#jetstarasia', '#singapore', '#staycation', ..."
898,yup got tank #nationalmuseumsg #repost bellael...,"['#nationalmuseumsg', '#repost', '#nationalmus..."
1034,back turn staycation vibes fulfil summer daydr...,"['#fourpointssingapore', '#fourpointssg', '#fo..."
...,...,...
14358,take break hectic everyday soothed sights soun...,"['', '#safesingapore']"
14370,rediscover family time away routine whether it...,"['', '#safesingapore']"
14441,heard sling know origin cocktailstep back time...,"['#tripflashsale', '#tripsg', '', '#rafflessin..."
14453,need ideas weekend find explore don’t forget p...,"['#safesingapore', '']"


In [77]:
# there are probably still more advertisements apart from these found
# but this is the most straightforward way to filter out the noise from advertising
# for example, there would probably still be advertising posts from (micro) influencers that are unverified

In [78]:
df_insta_sel = df_insta[~df_insta["owner_verified"]]
df_insta_sel.shape # shape of selected insta posts

(14256, 25)

In [79]:
df_insta_sel.reset_index(drop=True, inplace=True) # ensure index is reset

In [80]:
# save out merged df
df_insta_sel.to_csv("../datasets/df_insta_sel.csv", index=False)

### YouTube Videos

#### Georgia Caney Scrapes

In [86]:
df_yt_georgia = pd.read_csv("../datasets/df_yt_georgia_2020-10-17.csv")
df_yt_georgia.shape

(310, 8)

In [87]:
df_yt_georgia.head(3)

,video_title,video_caption,date_time,video_slug,video_views,video_likes,video_dislikes,video_comments
0,HALLOWEEN HORROR NIGHTS 8 AT UNIVERSAL STUDIOS! 👻,['UNIVERSAL HALLOWEEN HORROR NIGHTS 8! 👻\n\nHa...,31 Oct 2018,7xkdm3c4Ks8,"8,766 views",271,3,18 Comments
1,HUGE FEBRUARY HAUL | MONKI & ROMWE,"['FOLLOW ME!!\nBLOG LOVIN: ', 'http://www.blog...",28 Feb 2016,uBRHR9zK3mk,"16,049 views",312,5,NaN
2,Eating the BEST rated PRATA in SINGAPORE! 🇸🇬 *...,['Eating the BEST rated PRATA in SINGAPORE! 🇸🇬...,17 Nov 2019,b-ppBtSiG38,"33,483 views",649,19,159 Comments


In [88]:
# there are some duplicate scrapes
df_yt_georgia.duplicated(keep="first").value_counts()

False    302
True       8
dtype: int64

In [89]:
# drop duplicate scrapes
df_yt_georgia = df_yt_georgia.drop(df_yt_georgia[df_yt_georgia.duplicated(keep="first")].index)
df_yt_georgia.shape

(302, 8)

In [90]:
df_yt_georgia.info() # some missing values

<class 'pandas.core.frame.DataFrame'>
Int64Index: 302 entries, 0 to 309
Data columns (total 8 columns):
 #   Column          Non-Null Count  Dtype 
---  ------          --------------  ----- 
 0   video_title     301 non-null    object
 1   video_caption   301 non-null    object
 2   date_time       301 non-null    object
 3   video_slug      301 non-null    object
 4   video_views     301 non-null    object
 5   video_likes     299 non-null    object
 6   video_dislikes  299 non-null    object
 7   video_comments  294 non-null    object
dtypes: object(8)
memory usage: 21.2+ KB


In [91]:
df_yt_georgia[df_yt_georgia["date_time"].isnull()] # found row(s) of missing values

,video_title,video_caption,date_time,video_slug,video_views,video_likes,video_dislikes,video_comments
27,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [92]:
# remove rows with all missing values
df_yt_georgia = df_yt_georgia.dropna(how="all").reset_index(drop=True)
df_yt_georgia.shape # verify 2 rows dropped

(301, 8)

#### Parse Dates

In [93]:
# unable to convert col of dates to datetime objs
# let's find the culprit
for date in df_yt_georgia["date_time"]:
    try:
        pd.to_datetime(date)
    except:
        print(date)

Premiered on 28 Aug 2019


In [94]:
mask = df_yt_georgia["date_time"] == "Premiered on 28 Aug 2019" # find culprit
df_yt_georgia[mask]

,video_title,video_caption,date_time,video_slug,video_views,video_likes,video_dislikes,video_comments
194,Singaporean Eats My Western Diet For The Day! ...,['Singaporean Eats My British Diet For The Day...,Premiered on 28 Aug 2019,pVwhDPWmJS0,"49,684 views",822,18,106 Comments


In [95]:
# video is relevant to our project
# unsure why date_time is of a different format from the usual but let's remedy it
df_yt_georgia.loc[mask, "date_time"] = re.findall("\d+\s\w+\s\d+", df_yt_georgia.loc[mask, "date_time"].values[0])[0]

In [96]:
df_yt_georgia["date_time"] = pd.to_datetime(df_yt_georgia["date_time"])
df_yt_georgia.info() # successfully converted date_time to datetime objs

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 301 entries, 0 to 300
Data columns (total 8 columns):
 #   Column          Non-Null Count  Dtype         
---  ------          --------------  -----         
 0   video_title     301 non-null    object        
 1   video_caption   301 non-null    object        
 2   date_time       301 non-null    datetime64[ns]
 3   video_slug      301 non-null    object        
 4   video_views     301 non-null    object        
 5   video_likes     299 non-null    object        
 6   video_dislikes  299 non-null    object        
 7   video_comments  294 non-null    object        
dtypes: datetime64[ns](1), object(7)
memory usage: 18.9+ KB


In [97]:
# there are still missing values in video_likes, video_dislikes, and video_comments
# video_views, video_likes, video_dislikes, and video_comments are all str objs when they should be numeric

#### Views and Likes

In [98]:
# extract no. of views, remove "," separator, and convert to int dtype
df_yt_georgia["video_views"] = df_yt_georgia["video_views"].str.extract(r"(\d+.\d+)", expand=False).str.replace(",", "").astype(int)
df_yt_georgia.info() # verify conversion is successful

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 301 entries, 0 to 300
Data columns (total 8 columns):
 #   Column          Non-Null Count  Dtype         
---  ------          --------------  -----         
 0   video_title     301 non-null    object        
 1   video_caption   301 non-null    object        
 2   date_time       301 non-null    datetime64[ns]
 3   video_slug      301 non-null    object        
 4   video_views     301 non-null    int32         
 5   video_likes     299 non-null    object        
 6   video_dislikes  299 non-null    object        
 7   video_comments  294 non-null    object        
dtypes: datetime64[ns](1), int32(1), object(6)
memory usage: 17.8+ KB


In [99]:
mask = df_yt_georgia["video_likes"].isnull() # find missing values
df_yt_georgia[mask]

,video_title,video_caption,date_time,video_slug,video_views,video_likes,video_dislikes,video_comments
80,TASTE TESTING SINGAPORE 7/11 DRINKS! 🇸🇬,"[""What drinks do they sell in a Singapore 7/11...",2017-11-12,3Q5Rh7gWc34,89417,NaN,NaN,0
239,THINGS TOOK A TURN FOR THE WORSE | WEEKLY VLOG,['THINGS TOOK A TURN FOR THE WORSE IN SINGAPOR...,2018-08-16,72oE5ABnd4k,19886,NaN,NaN,0


In [100]:
# checking via slugs of the video revealed that these videos have zero likes or dislikes
# while this seems especially unlikely because of the video's recency
# comments for both these videos were also disabled
# it is likely that likes/dislikes were disabled too

# let's impute missing values in video_likes and video_dislikes with 0
df_yt_georgia.loc[:, ["video_likes", "video_dislikes"]] = df_yt_georgia.loc[:, ["video_likes", "video_dislikes"]].fillna("0")
df_yt_georgia.info() # verify that missing values have been imputed

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 301 entries, 0 to 300
Data columns (total 8 columns):
 #   Column          Non-Null Count  Dtype         
---  ------          --------------  -----         
 0   video_title     301 non-null    object        
 1   video_caption   301 non-null    object        
 2   date_time       301 non-null    datetime64[ns]
 3   video_slug      301 non-null    object        
 4   video_views     301 non-null    int32         
 5   video_likes     301 non-null    object        
 6   video_dislikes  301 non-null    object        
 7   video_comments  294 non-null    object        
dtypes: datetime64[ns](1), int32(1), object(6)
memory usage: 17.8+ KB


In [101]:
# extract no. of likes/dislikes, remove "," separator, and convert to int dtype
df_yt_georgia["video_likes"] = df_yt_georgia["video_likes"].str.replace(",", "").astype(int)
df_yt_georgia["video_dislikes"] = df_yt_georgia["video_dislikes"].str.replace(",", "").astype(int)
df_yt_georgia.info() # verify conversion is successful

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 301 entries, 0 to 300
Data columns (total 8 columns):
 #   Column          Non-Null Count  Dtype         
---  ------          --------------  -----         
 0   video_title     301 non-null    object        
 1   video_caption   301 non-null    object        
 2   date_time       301 non-null    datetime64[ns]
 3   video_slug      301 non-null    object        
 4   video_views     301 non-null    int32         
 5   video_likes     301 non-null    int32         
 6   video_dislikes  301 non-null    int32         
 7   video_comments  294 non-null    object        
dtypes: datetime64[ns](1), int32(3), object(4)
memory usage: 15.4+ KB


#### No. of Comments

In [102]:
df_yt_georgia["video_comments"]

0       18 Comments
1               NaN
2      159 Comments
3      117 Comments
4                 0
           ...     
296    140 Comments
297     62 Comments
298    217 Comments
299    145 Comments
300             NaN
Name: video_comments, Length: 301, dtype: object

In [103]:
# remove all "Comments" text, "," for the thousandth place, and strip spaces
df_yt_georgia.loc[:, "video_comments"] = df_yt_georgia["video_comments"].str.replace("Comments", "").str.replace(",", "").str.strip()

In [104]:
mask = df_yt_georgia["video_comments"].isnull() # find missing values
print("No. of posts with missing total no. of video comments:", len(df_yt_georgia[mask]))
df_yt_georgia[mask]

No. of posts with missing total no. of video comments: 7


,video_title,video_caption,date_time,video_slug,video_views,video_likes,video_dislikes,video_comments
1,HUGE FEBRUARY HAUL | MONKI & ROMWE,"['FOLLOW ME!!\nBLOG LOVIN: ', 'http://www.blog...",2016-02-28,uBRHR9zK3mk,16049,312,5,NaN
12,"VIRTUAL WISHLIST | EBAY, URBAN OUTFITTERS, AN...","['FOLLOW ME!!\nBLOG LOVIN: ', 'http://www.blog...",2015-02-27,GFZO5_uXnzo,6864,413,6,NaN
138,TRYING TO PRONOUNCE SINGAPORE MRT STATIONS!,"[""We finally got around to ATTEMPTING to prono...",2017-05-28,bIgaCSZn_7U,32296,715,1066,NaN
162,HUGE ASOS TRY-ON HAUL! | NOVEMBER 2016,"['➫ Instagram: ', 'https://goo.gl/EG0bLO', '\n...",2016-11-27,lNtAfcNmrD4,7851,211,6,NaN
172,COME TO MY FIRST PERSONAL TRAINING SESSION! *...,['Hey guys! Today I wanted to take you all alo...,2019-03-07,FGSISjGXUUk,11671,227,6,NaN
197,HOMEWARE HAUL! IKEA & TYPO THE VINTAGE VISION,['Help me share this post & I will love you fo...,2015-05-18,Oi65C3XnCz0,10094,320,4,NaN
300,MY FOUNDATION & CONTOUR ROUTINE | THE VINTAGE ...,"['FOLLOW ME!!\nBLOG LOVIN: ', 'http://www.blog...",2015-02-19,7t89IwQ90Lg,6004,236,4,NaN


In [105]:
# small no. of videos with this problem, perhaps comments were disabled for these videos
# let's just imput with 0
df_yt_georgia.loc[:, "video_comments"] = df_yt_georgia["video_comments"].fillna(0)

In [106]:
# convert col to int dtype
df_yt_georgia.loc[:, "video_comments"] = df_yt_georgia["video_comments"].astype(int)
df_yt_georgia.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 301 entries, 0 to 300
Data columns (total 8 columns):
 #   Column          Non-Null Count  Dtype         
---  ------          --------------  -----         
 0   video_title     301 non-null    object        
 1   video_caption   301 non-null    object        
 2   date_time       301 non-null    datetime64[ns]
 3   video_slug      301 non-null    object        
 4   video_views     301 non-null    int32         
 5   video_likes     301 non-null    int32         
 6   video_dislikes  301 non-null    int32         
 7   video_comments  301 non-null    int32         
dtypes: datetime64[ns](1), int32(4), object(3)
memory usage: 14.2+ KB


#### Merging Text

In [107]:
# before merging, let's clean the text
df_yt_georgia["video_title_clean"] = clean_text(df_yt_georgia, "video_title")
df_yt_georgia["video_caption_clean"] = clean_text(df_yt_georgia, "video_caption")
df_yt_georgia.head(1)

,video_title,video_caption,date_time,video_slug,video_views,video_likes,video_dislikes,video_comments,video_title_clean,video_caption_clean
0,HALLOWEEN HORROR NIGHTS 8 AT UNIVERSAL STUDIOS! 👻,['UNIVERSAL HALLOWEEN HORROR NIGHTS 8! 👻\n\nHa...,2018-10-31,7xkdm3c4Ks8,8766,271,3,18,halloween horror nights 8 universal studios,'universal halloween horror nights 8 happy hal...


In [108]:
content = [] # create empty list

for _, row in df_yt_georgia.iterrows():
    content.append(row["video_title_clean"] + " " + row["video_caption_clean"]) # join title and caption strings
    
df_yt_georgia["content"] = content # create new col for merged strings
df_yt_georgia.shape # verify new col added

(301, 11)

In [109]:
df_yt_georgia.head(3)

,video_title,video_caption,date_time,video_slug,video_views,video_likes,video_dislikes,video_comments,video_title_clean,video_caption_clean,content
0,HALLOWEEN HORROR NIGHTS 8 AT UNIVERSAL STUDIOS! 👻,['UNIVERSAL HALLOWEEN HORROR NIGHTS 8! 👻\n\nHa...,2018-10-31,7xkdm3c4Ks8,8766,271,3,18,halloween horror nights 8 universal studios,'universal halloween horror nights 8 happy hal...,halloween horror nights 8 universal studios 'u...
1,HUGE FEBRUARY HAUL | MONKI & ROMWE,"['FOLLOW ME!!\nBLOG LOVIN: ', 'http://www.blog...",2016-02-28,uBRHR9zK3mk,16049,312,5,0,huge february haul monki romwe,'follow blog lovin ' ' ' blog ' ' ' depop geor...,huge february haul monki romwe 'follow blog lo...
2,Eating the BEST rated PRATA in SINGAPORE! 🇸🇬 *...,['Eating the BEST rated PRATA in SINGAPORE! 🇸🇬...,2019-11-17,b-ppBtSiG38,33483,649,19,159,eating best rated prata longest queue ever,'eating best rated prata eaten prata think che...,eating best rated prata longest queue ever 'ea...


#### Selecting Relevant Videos

In [110]:
# before selecting, let's save out cleaned GC df
df_yt_georgia.to_csv("../datasets/df_yt_georgia_cleaned.csv", index=False)

In [111]:
# what are the relevant videos?

In [112]:
df_yt_georgia["video_title"].values[:35]

array(['HALLOWEEN HORROR NIGHTS 8 AT UNIVERSAL STUDIOS! 👻',
       'HUGE FEBRUARY HAUL | MONKI &  ROMWE',
       'Eating the BEST rated PRATA in SINGAPORE! 🇸🇬 *longest queue ever*',
       'McDoanlds Hokkaido Salmon Burger In Singapore!',
       'SINGAPORE APARTMENT TOUR! | THE VINTAGE VISION',
       'Foreigners Reacting To Singapore Social: The Most NON-SINGAPOREAN Reality Show',
       'THAILAND ADVENTURE & MOPED ACCIDENT!!',
       'CULTURE SHOCKS LIVING IN SINGAPORE! 🇸🇬',
       'Taking it slow + saying goodbye to the vlogs | Life in Singapore',
       'HUGE SUMMER TRY-ON HAUL! TOPSHOP, ZARA, ASOS',
       'More Bad News, But Trying To Keep Positive! Life In Singapore',
       'MISCONCEPTIONS FOREIGNERS HAVE ABOUT SINGAPORE! 🇸🇬',
       'VIRTUAL WISHLIST |  EBAY, URBAN OUTFITTERS, ANTHROPOLOGIE',
       'COME TO WORK WITH ME IN SINGAPORE!',
       'BRITISH GIRL REACTS TO SINGAPOREAN YOUTUBERS!',
       'One Bedroom Singapore Apartment Tour! Georgia Caney',
       'Reading Your Ass

After studying the video titles and researching the YouTube channel, it is clear that there is a good part of her videos dedicated to fashion and e-commerce. There are also videos about her life updates that do not concern exploring Singapore and which mostly elicit only empathetic/agreeable comments rather than the discourse and local recommendations we are looking for. Then, there are also travel vlogs to numerous locations.

We shall begin by filtering these 3 categories out as they are not relevant to domestic tourism. We shall do so by querying select key words that should help us filter out the videos that we do not want.

In [114]:
# search terms to find videos dedicated to fashion/e-commerce
queries = ["haul", "the vintage vision", "wishlist", "outfits", "lookbook", "discount", "dupes", "shopping",
           "favourites", "makeup", "hair", "lip", "jeans", "contour", "thrift", "handbag", "swatches", "glasses"]
fashion = df_yt_georgia["video_title"].str.lower().str.contains("|".join(queries))
df_yt_georgia[fashion].shape # 77 fashion videos found

(91, 11)

In [115]:
# search terms to find videos dedicated to life updates
queries = ["life in singapore", "life currently in singapore", "catch up", "expat life", "workout",
           "questions", "answers", "q&a", "diet", "update", "weekly", "instagram"]
updates = df_yt_georgia["video_title"].str.lower().str.contains("|".join(queries))
df_yt_georgia[updates].shape # 11 update videos found

(51, 11)

In [116]:
# search terms to find videos dedicated to travel
queries = ["australia", "thailand", "uk", "🇬🇧", "penang", "kuala lumpur", "malaysia", "bali", "borneo", "cambodia", "taipei"]
travel = df_yt_georgia["video_title"].str.lower().str.contains("|".join(queries))
df_yt_georgia[travel].shape # 11 update videos found

(28, 11)

In [117]:
# what are we left with after filtering these out?
df_yt_georgia_sel = df_yt_georgia[~fashion & ~updates & ~travel]
df_yt_georgia_sel.shape # down to 134 posts

(134, 11)

In [118]:
# view current df
df_yt_georgia_sel.head(3)

,video_title,video_caption,date_time,video_slug,video_views,video_likes,video_dislikes,video_comments,video_title_clean,video_caption_clean,content
0,HALLOWEEN HORROR NIGHTS 8 AT UNIVERSAL STUDIOS! 👻,['UNIVERSAL HALLOWEEN HORROR NIGHTS 8! 👻\n\nHa...,2018-10-31,7xkdm3c4Ks8,8766,271,3,18,halloween horror nights 8 universal studios,'universal halloween horror nights 8 happy hal...,halloween horror nights 8 universal studios 'u...
2,Eating the BEST rated PRATA in SINGAPORE! 🇸🇬 *...,['Eating the BEST rated PRATA in SINGAPORE! 🇸🇬...,2019-11-17,b-ppBtSiG38,33483,649,19,159,eating best rated prata longest queue ever,'eating best rated prata eaten prata think che...,eating best rated prata longest queue ever 'ea...
3,McDoanlds Hokkaido Salmon Burger In Singapore!,['TRYING MCDONALDS HOKKAIDO SALMON BURGER IN S...,2018-05-13,IDXoNb548ec,13809,430,13,117,mcdoanlds hokkaido salmon burger,'trying mcdonalds hokkaido salmon burger try i...,mcdoanlds hokkaido salmon burger 'trying mcdon...


#### Ghib Ojisan Scrapes

In [119]:
df_yt_ghib = pd.read_csv("../datasets/df_yt_ghib_2020-10-18.csv")
df_yt_ghib.shape

(389, 8)

In [120]:
df_yt_ghib.head(3)

,video_title,video_caption,date_time,video_slug,video_views,video_likes,video_dislikes,video_comments
0,【これが現実】エリート駐在員にストレスを吐き出してもらった結果…【シンガポール】,"['▼基本一人旅さん\n', 'https://twitter.com/crazy_trav...",7 Jun 2019,AsF1hDfyF9g,"26,426 views",347,36,77 Comments
1,【発表】路上演奏のライセンスを取りにオーディションに参加した結果,"['😃チャンネル登録： ', 'http://urx3.nu/HTUJ', '\n🎥関連動画...",31 Jul 2019,R4rwlGx0Kog,"10,547 views",405,6,102 Comments
2,【裏側公開】ビートルズのノルウェーの森を弾いてみた【メイキング動画】,['以前、『ノルウェーの森でノルウェーの森を弾いてみた』という（詰まらないシャレみたいな）動...,12 Aug 2018,0oMslC1sGNU,"10,514 views",258,7,62 Comments


In [121]:
# there are some duplicate scrapes
df_yt_ghib.duplicated(keep="first").value_counts()

False    378
True      11
dtype: int64

In [122]:
# drop duplicate scrapes
df_yt_ghib = df_yt_ghib.drop(df_yt_ghib[df_yt_ghib.duplicated(keep="first")].index)
df_yt_ghib.shape

(378, 8)

In [123]:
df_yt_ghib.info() # some missing values

<class 'pandas.core.frame.DataFrame'>
Int64Index: 378 entries, 0 to 388
Data columns (total 8 columns):
 #   Column          Non-Null Count  Dtype 
---  ------          --------------  ----- 
 0   video_title     377 non-null    object
 1   video_caption   377 non-null    object
 2   date_time       377 non-null    object
 3   video_slug      377 non-null    object
 4   video_views     377 non-null    object
 5   video_likes     377 non-null    object
 6   video_dislikes  377 non-null    object
 7   video_comments  374 non-null    object
dtypes: object(8)
memory usage: 26.6+ KB


In [124]:
df_yt_ghib[df_yt_ghib["date_time"].isnull()] # found row(s) of missing values

,video_title,video_caption,date_time,video_slug,video_views,video_likes,video_dislikes,video_comments
15,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [125]:
# remove rows with all missing values
df_yt_ghib = df_yt_ghib.dropna(how="all").reset_index(drop=True)
df_yt_ghib.shape # verify 2 rows dropped

(377, 8)

#### Parse Dates

In [126]:
# unable to convert col of dates to datetime objs
# let's find the culprits
culprits = [] # create empty lists to append culprits

for date in df_yt_ghib["date_time"]:
    try:
        pd.to_datetime(date)
    except:
        culprits.append(date)
        
culprits

['Streamed live on 20 Nov 2019',
 'Streamed live on 7 Aug 2020',
 'Streamed live on 22 Jul 2020',
 'Streamed live on 10 Feb 2019',
 'Streamed live on 14 Oct 2018',
 'Streamed live on 9 Jul 2020',
 'Streamed live on 4 Nov 2018',
 'Streamed live on 21 Aug 2020',
 'Premiered on 26 Oct 2018',
 'Streamed live on 13 Jan 2019']

In [127]:
mask = df_yt_ghib["date_time"].isin(culprits) # find culprit
df_yt_ghib[mask]

,video_title,video_caption,date_time,video_slug,video_views,video_likes,video_dislikes,video_comments
97,登録者10万人突破記念！ギター演奏＆心境雑談＆質問コーナー【ありがとう】,['いつも見ていただいている皆さんのおかげで登録者10万人を突破しました！YouTubeを初...,Streamed live on 20 Nov 2019,tgmlrcmulEM,"10,912 views",499,8,25 Comments
98,LIVE | Which Singapore Bak Kut Teh is the BEST?,"[""Tonight, we will be trying Singapore's iconi...",Streamed live on 7 Aug 2020,d7z9g-0QH0g,"36,776 views",633,19,86 Comments
126,LIVE｜Trying 3 BEST Chili Crabs in Singapore - ...,"[""Today's a special collab stream with Singapo...",Streamed live on 22 Jul 2020,2Eiqtrbo3l4,"69,545 views","1,099",46,126 Comments
148,シンガポールから生配信！ご飯食べながら質問に答えます（アーカイブ）,['2019年2月11日に配信した動画のアーカイブ放送です。\nご覧いただいた方、スーパーチ...,Streamed live on 10 Feb 2019,NS-ng6ot9a8,"11,889 views",230,9,27 Comments
253,日曜日の夜にアコギ聴きませんか？【雑談・Q&Aも】,['初生配信です！\n\n・リクエスト受け付けます（出来るとは言ってない）\n・雑談\n・Q...,Streamed live on 14 Oct 2018,o9xj8E5_DsE,"11,276 views",488,8,52 Comments
322,Which Chicken Rice is the Best? Collab Stream ...,"[""Today's a special collab stream with Singapo...",Streamed live on 9 Jul 2020,EsayJ7_I4ec,"26,330 views",675,31,63 Comments
336,【シンガポールから配信】日曜の夜に雑談でもしませんか？,['今日は演奏少なめ・雑談多めというコンセプトでやります！\nでもリクエストも受け付けます。...,Streamed live on 4 Nov 2018,k9OMEfEbkDE,"9,049 views",230,10,20 Comments
345,LIVE | Which Singapore Hokkien Mee Is The Best...,"[""Tonight, me and a SECRET guest will be tryin...",Streamed live on 21 Aug 2020,DIHL0k6mI8s,"25,644 views",674,21,71 Comments
361,【笑う】ツッコミどころが多すぎるギター演奏をご覧ください,['前使ってたパソコンから秘蔵映像を見つけたので、自分でネタにしてみましたｗ\n\nギター、...,Premiered on 26 Oct 2018,4JaVNeALvlE,"28,882 views",456,32,112 Comments
368,いい質問してくれたら1曲プレゼント！ギター×世界一周、海外移住、ジブリなどの質問に答えるよ,['※1/13の配信です。\n\n海外の路上でジブリ弾きながら世界旅行したジブリおじさんです...,Streamed live on 13 Jan 2019,4UF_X0PaDDg,"7,898 views",190,10,52 Comments


In [128]:
# let's extract out the dates so we can parse col as such
df_yt_ghib.loc[mask, "date_time"] = re.findall("\d+\s\w+\s\d+", df_yt_ghib.loc[mask, "date_time"].values[0])[0]

In [129]:
df_yt_ghib["date_time"] = pd.to_datetime(df_yt_ghib["date_time"])
df_yt_ghib.info() # successfully converted date_time to datetime objs

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 377 entries, 0 to 376
Data columns (total 8 columns):
 #   Column          Non-Null Count  Dtype         
---  ------          --------------  -----         
 0   video_title     377 non-null    object        
 1   video_caption   377 non-null    object        
 2   date_time       377 non-null    datetime64[ns]
 3   video_slug      377 non-null    object        
 4   video_views     377 non-null    object        
 5   video_likes     377 non-null    object        
 6   video_dislikes  377 non-null    object        
 7   video_comments  374 non-null    object        
dtypes: datetime64[ns](1), object(7)
memory usage: 23.7+ KB


In [130]:
# there are still missing values in video_comments
# video_views, video_likes, video_dislikes, and video_comments are all str objs when they should be numeric

#### Views and Likes

In [131]:
df_yt_ghib["video_likes"]

0        347
1        405
2        258
3      3,155
4      1,931
       ...  
372    2,153
373      424
374    1,556
375    1,002
376    1,460
Name: video_likes, Length: 377, dtype: object

In [132]:
# extract no. of views, remove "," separator, and convert to int dtype
df_yt_ghib["video_views"] = df_yt_ghib["video_views"].str.extract(r"(\d+.\d+)", expand=False).str.replace(",", "").astype(int)
df_yt_ghib.info() # verify conversion is successful

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 377 entries, 0 to 376
Data columns (total 8 columns):
 #   Column          Non-Null Count  Dtype         
---  ------          --------------  -----         
 0   video_title     377 non-null    object        
 1   video_caption   377 non-null    object        
 2   date_time       377 non-null    datetime64[ns]
 3   video_slug      377 non-null    object        
 4   video_views     377 non-null    int32         
 5   video_likes     377 non-null    object        
 6   video_dislikes  377 non-null    object        
 7   video_comments  374 non-null    object        
dtypes: datetime64[ns](1), int32(1), object(6)
memory usage: 22.2+ KB


In [133]:
# extract no. of likes/dislikes, remove "," separator, and convert to int dtype
df_yt_ghib["video_likes"] = df_yt_ghib["video_likes"].str.replace(",", "").astype(int)
df_yt_ghib["video_dislikes"] = df_yt_ghib["video_dislikes"].str.replace(",", "").astype(int)
df_yt_ghib.info() # verify conversion is successful

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 377 entries, 0 to 376
Data columns (total 8 columns):
 #   Column          Non-Null Count  Dtype         
---  ------          --------------  -----         
 0   video_title     377 non-null    object        
 1   video_caption   377 non-null    object        
 2   date_time       377 non-null    datetime64[ns]
 3   video_slug      377 non-null    object        
 4   video_views     377 non-null    int32         
 5   video_likes     377 non-null    int32         
 6   video_dislikes  377 non-null    int32         
 7   video_comments  374 non-null    object        
dtypes: datetime64[ns](1), int32(3), object(4)
memory usage: 19.3+ KB


#### No. of Comments

In [134]:
df_yt_ghib["video_comments"]

0       77 Comments
1      102 Comments
2       62 Comments
3      172 Comments
4      318 Comments
           ...     
372    288 Comments
373     57 Comments
374    152 Comments
375    115 Comments
376             NaN
Name: video_comments, Length: 377, dtype: object

In [135]:
# remove all "Comments" text, "," for the thousandth place, and strip spaces
df_yt_ghib.loc[:, "video_comments"] = df_yt_ghib["video_comments"].str.replace("Comments", "").str.replace(",", "").str.strip()

In [136]:
mask = df_yt_ghib["video_comments"].isnull() # find missing values
print("No. of posts with missing total no. of video comments:", len(df_yt_ghib[mask]))
df_yt_ghib[mask]

No. of posts with missing total no. of video comments: 3


,video_title,video_caption,date_time,video_slug,video_views,video_likes,video_dislikes,video_comments
103,Singaporean Auntie vs Durian Seller👩🏻 Ultimat...,"[""Today you will witness the ultimate match - ...",2020-09-03,GN0mKg5EUts,71912,1335,50,NaN
149,A Day in a Life of a Japanese YouTuber in Sing...,"['TODAY, an online media company, contacted me...",2019-12-26,zF8vFCuilXQ,36986,1534,10,NaN
376,ロンドンで一日観光で遊び尽くしたらいくらかかる？,['▼海外専門のグルメチャンネル「タベルトラベル」開設しました！日本人YouTuberで唯一...,2018-09-01,lr6gSHXhgxw,98613,1460,82,NaN


In [137]:
# small no. of videos with this problem, perhaps comments were disabled for these videos
# let's just imput with 0
df_yt_ghib.loc[:, "video_comments"] = df_yt_ghib["video_comments"].fillna(0)

In [138]:
# convert col to int dtype
df_yt_ghib.loc[:, "video_comments"] = df_yt_ghib["video_comments"].astype(int)
df_yt_ghib.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 377 entries, 0 to 376
Data columns (total 8 columns):
 #   Column          Non-Null Count  Dtype         
---  ------          --------------  -----         
 0   video_title     377 non-null    object        
 1   video_caption   377 non-null    object        
 2   date_time       377 non-null    datetime64[ns]
 3   video_slug      377 non-null    object        
 4   video_views     377 non-null    int32         
 5   video_likes     377 non-null    int32         
 6   video_dislikes  377 non-null    int32         
 7   video_comments  377 non-null    int32         
dtypes: datetime64[ns](1), int32(4), object(3)
memory usage: 17.8+ KB


#### Merging Text

In [139]:
# before merging, let's clean the text
df_yt_ghib["video_title_clean"] = clean_text(df_yt_ghib, "video_title")
df_yt_ghib["video_caption_clean"] = clean_text(df_yt_ghib, "video_caption")
df_yt_ghib.head(1)

,video_title,video_caption,date_time,video_slug,video_views,video_likes,video_dislikes,video_comments,video_title_clean,video_caption_clean
0,【これが現実】エリート駐在員にストレスを吐き出してもらった結果…【シンガポール】,"['▼基本一人旅さん\n', 'https://twitter.com/crazy_trav...",2019-06-07,AsF1hDfyF9g,26426,347,36,77,…,' ' ' ' … … ' ' ' ' ' ' ' ' ' ' '#' 'u3000' '#...


In [140]:
content = [] # create empty list

for _, row in df_yt_ghib.iterrows():
    content.append(row["video_title"] + " " + row["video_caption"]) # join title and caption strings
    
df_yt_ghib["content"] = content # create new col for merged strings
df_yt_ghib.shape # verify new col added

(377, 11)

In [141]:
df_yt_ghib.head(1)

,video_title,video_caption,date_time,video_slug,video_views,video_likes,video_dislikes,video_comments,video_title_clean,video_caption_clean,content
0,【これが現実】エリート駐在員にストレスを吐き出してもらった結果…【シンガポール】,"['▼基本一人旅さん\n', 'https://twitter.com/crazy_trav...",2019-06-07,AsF1hDfyF9g,26426,347,36,77,…,' ' ' ' … … ' ' ' ' ' ' ' ' ' ' '#' 'u3000' '#...,【これが現実】エリート駐在員にストレスを吐き出してもらった結果…【シンガポール】 ['▼基本...


#### Selecting Relevant Videos

In [142]:
# before selecting, let's save out cleaned GO df
df_yt_ghib.to_csv("../datasets/df_yt_ghib_cleaned.csv", index=False)

In [143]:
# what are the relevant videos?

In [144]:
df_yt_ghib["video_title"].values[:35]

array(['【これが現実】エリート駐在員にストレスを吐き出してもらった結果…【シンガポール】',
       '【発表】路上演奏のライセンスを取りにオーディションに参加した結果',
       '【裏側公開】ビートルズのノルウェーの森を弾いてみた【メイキング動画】',
       'LAP TAPPING Guitar at Taipei!\u3000台北でひざ置きギター弾いてみた（Jack Dawson & Andy Mckee "Drifting" Cover）',
       'Singapore Semi-Lockdown Diary Day 0 | Japanese Living in SG',
       'How Much Will I Earn If I Played Hikoukigumo on the Street of Singapore? WIND RISES',
       '【天国】シンガポールの会社のリアルな福利厚生 事情',
       'Visiting The Last Surviving Village in Singapore | Kampong Lorong Buangkok🇸🇬',
       '外国人が連想する「日本」の本当のイメージとは？海外に20年住んだ僕が教える',
       'バンコクの大都会で路上ギターしたらチップいくら入る？【タイ】',
       "Japanese Reviews Singapore's UNI Restaurant | 🇸🇬Uni Gallery",
       'Hawkers After the Lock Down | Old Airport Road Food Centre, Singapore',
       'Cute Girl Tipping me! Busking at SPLIT, CROATIA. "All Laid Back and Stuff", composed by Andy Mckee,',
       'I Was Challenged By My Viewer… And This Is The Outcome🥊😤',
       'Japanese YouTuber Meets Dee Kosh | Famous Sing

In [145]:
df_yt_ghib["video_caption"].values[:35]

array(["['▼基本一人旅さん\\n', 'https://twitter.com/crazy_traveler1', '\\n\\n今回はシンガポールで駐在員として働く「基本一人旅」さんに駐在員の闇について迫りました。\\n\\n当然、働き方や待遇は会社によりけりですが、話を聞く限りめちゃくちゃ大変そう…。\\n\\n海外の駐在員は\\n・いい家\\n・健康保険\\n・高い給料\\n\\nなどを会社が提供してくれますが、その引き換えに\\n\\n・いつ来るか分からない任期\\n・会社に縛れる（圧倒的なコミットを強制される）ストレス\\n\\nこれらに苦しんでいる方が多いイメージです。\\n\\n一見、華やかな見える駐在員の生活ですが、今回お話聞いて「一概には言えないんだな」と感じましたね〜…。\\n\\nちなみに、現地採用のぼくは割とゆるくやらせてもらっています😇\\n\\n前回の動画：\\n・シンガポール駐在員の自宅を大公開。家賃は？\\n', 'https://www.youtube.com/watch?v=JJNW7...', '\\n・シンガポール駐在員の給料について暴露\\n', 'https://www.youtube.com/watch?v=p4DrC...', '\\n\\nシンガポールで就職する方法：\\n', 'https://www.ghib-oji.com/singapore-jo...', '\\n\\n\\n', '#シンガポール', '\\u3000', '#現地採用', '\\u3000', '#駐在員', '\\u3000', '#生活', '\\n━━━━━━━━━━━━\\n✔️チャンネル登録！（subscribe)\\n ', 'http://urx3.nu/HTUJ', '\\n━━━━━━━━━━━━\\n🌐SNS・ブログ\\n・Instagram → ', 'https://www.instagram.com/ghibli_ojisan/', '\\n・Twitter → ', 'https://twitter.com/ghibli_ojisan', '\\n・Blog→ ', 'https://www.ghib-oji.com/', '\\n━━━━━━━━━━━━\\n🎸おすすめ動画3選！\\n・台湾・九份で

After studying the video titles and researching the YouTube channel, the first videos to filter out will be all the Japanese-titled videos that cater to the Japanese-speaking community. We will filter them out since our scope of our NLP project deals only with English word features.

In [147]:
df_yt_ghib["video_title"][0]

'【これが現実】エリート駐在員にストレスを吐き出してもらった結果…【シンガポール】'

In [148]:
re.findall("[A-Za-z]", df_yt_ghib["video_title"][0])

[]

In [149]:
# use regex to extract out all english characters
# if video title is entirely in japanese, no english characters will be extracted
# filtering by an .notnull() method later will allow us to find all videos with (partial) english titles
df_yt_ghib["video_eng"] = df_yt_ghib["video_title"].str.extractall("([A-Za-z]+)").groupby(level=0)[0].apply(lambda x: " ".join(x))
df_yt_ghib.shape # verify new col added

(377, 12)

In [150]:
df_yt_ghib_eng = df_yt_ghib[df_yt_ghib["video_eng"].notnull()]
df_yt_ghib_eng["video_title"].values

array(['LAP TAPPING Guitar at Taipei!\u3000台北でひざ置きギター弾いてみた（Jack Dawson & Andy Mckee "Drifting" Cover）',
       'Singapore Semi-Lockdown Diary Day 0 | Japanese Living in SG',
       'How Much Will I Earn If I Played Hikoukigumo on the Street of Singapore? WIND RISES',
       'Visiting The Last Surviving Village in Singapore | Kampong Lorong Buangkok🇸🇬',
       "Japanese Reviews Singapore's UNI Restaurant | 🇸🇬Uni Gallery",
       'Hawkers After the Lock Down | Old Airport Road Food Centre, Singapore',
       'Cute Girl Tipping me! Busking at SPLIT, CROATIA. "All Laid Back and Stuff", composed by Andy Mckee,',
       'I Was Challenged By My Viewer… And This Is The Outcome🥊😤',
       'Japanese YouTuber Meets Dee Kosh | Famous Singapore YouTuber🇸🇬',
       'A Day in My Life as a Traveling Busker | Taiwan🇹🇼',
       'I Have Never Been So Disappointed 🇸🇬Holland Village',
       '13 AMAZING Things to See and Do at Singapore Changi Airport [Japanese]',
       'Would You Try This? Going Too Deep

In [151]:
df_yt_ghib_eng.shape

(257, 12)

Upon a closer look at the remaining posts, there is a need for us to be stricter with our videos selection. There are quite a number of posts that are still in Japanese that were not filtered out because of there are English words that supplement the mainly Japanese title/descriptions. Let us filter these out first.

In [153]:
# but first, let's drop the col created above since it has no use anymore
df_yt_ghib.drop("video_eng", axis=1, inplace=True)

In [154]:
# we can use the .isascii() method to filter out english and non-english text
# but .isascii() method does not capture certain videos that we want because of some non-ascii characters in the str
### these are "’" and "🇸🇬"

print(df_yt_ghib["video_title"][360]) # sample of .isascii() exception thrown
df_yt_ghib["video_title"][360].isascii()

How It’s Like Joining a LION DANCE Troupe in Singapore!🇸🇬


False

In [155]:
# let's replace them
df_yt_ghib.loc[:, ["video_title"]] = [s.replace("’", "'").replace("🇸🇬", "") for s in df_yt_ghib["video_title"]]
print(df_yt_ghib["video_title"][360]) # sample of replacement

How It's Like Joining a LION DANCE Troupe in Singapore!


In [156]:
# filter out all posts with japanese characters
only_eng = [title.isascii() for title in df_yt_ghib["video_title"]]
df_yt_ghib_eng = df_yt_ghib[only_eng] # rewrite df
df_yt_ghib_eng.shape

(145, 11)

In [157]:
df_yt_ghib_eng["video_title"].values

array(['Singapore Semi-Lockdown Diary Day 0 | Japanese Living in SG',
       'How Much Will I Earn If I Played Hikoukigumo on the Street of Singapore? WIND RISES',
       'Visiting The Last Surviving Village in Singapore | Kampong Lorong Buangkok',
       "Japanese Reviews Singapore's UNI Restaurant | Uni Gallery",
       'Hawkers After the Lock Down | Old Airport Road Food Centre, Singapore',
       'Cute Girl Tipping me! Busking at SPLIT, CROATIA. "All Laid Back and Stuff", composed by Andy Mckee,',
       'Japanese YouTuber Meets Dee Kosh | Famous Singapore YouTuber',
       'I Have Never Been So Disappointed Holland Village',
       '13 AMAZING Things to See and Do at Singapore Changi Airport [Japanese]',
       "You Wouldn't Believe What's Inside This Box!",
       "Inside Extreme Wow Suite | W Singapore's MOST EXPENSIVE ROOM!",
       'What I Like and Always Missed About Japan | Japanese Guy Goes Home',
       'KAONASHI (No-Face) Happily Plays Totoro at a Luxury in Singapore | W 

Finally, there are a number of travel videos as well as videos about guitar and/or busking. We shall also filter these out.

In [159]:
# search terms to find videos dedicated to travel
travel = ["croatia", "slovenia", "norway", "italy", "taiwan", "kyoto", "hokkaido", "osaka"]
travel = df_yt_ghib_eng["video_title"].str.lower().str.contains("|".join(travel))
df_yt_ghib_eng[travel].shape # 15 travel videos found

(16, 11)

In [160]:
# search terms to find videos dedicated to guitar/busking
busk = ["guitar", "busk", "tapping", "played by", "ghibli"]
busk = df_yt_ghib_eng["video_title"].str.lower().str.contains("|".join(busk))
df_yt_ghib_eng[busk].shape # 20 guitar/busking videos found

(20, 11)

In [161]:
# what are we left with after filtering these out?
df_yt_ghib_sel = df_yt_ghib_eng[~travel & ~busk]
df_yt_ghib_sel.shape # down to 70 posts

(114, 11)

In [162]:
# view current df
df_yt_ghib_sel.head(3)

,video_title,video_caption,date_time,video_slug,video_views,video_likes,video_dislikes,video_comments,video_title_clean,video_caption_clean,content
4,Singapore Semi-Lockdown Diary Day 0 | Japanese...,"[""This thing called the 'circuit breaker' is h...",2020-04-07,632WZ2a-GCw,84167,1931,51,318,semilockdown diary day 0 japanese living,thing called 'circuit breaker' happening actua...,Singapore Semi-Lockdown Diary Day 0 | Japanese...
5,How Much Will I Earn If I Played Hikoukigumo o...,"['😃Subscribe： ', 'http://urx3.nu/HTUJ', '\n\nI...",2019-08-05,PkVmO44f6p4,26635,550,11,92,much earn played hikoukigumo street wind rises,'subscribe ' ' ' played hikoukigumo ghibli fil...,How Much Will I Earn If I Played Hikoukigumo o...
7,Visiting The Last Surviving Village in Singapo...,"[""Enable CC for English subtitles!\nSo today I...",2019-12-20,YXb5tJdIwZs,59869,1501,23,249,visiting last surviving village kampong lorong...,enable cc english subtitles today visited kamp...,Visiting The Last Surviving Village in Singapo...


#### Export Selected Videos

In [163]:
print(df_yt_georgia_sel.shape)
df_yt_georgia_sel.head(1)

(134, 11)


,video_title,video_caption,date_time,video_slug,video_views,video_likes,video_dislikes,video_comments,video_title_clean,video_caption_clean,content
0,HALLOWEEN HORROR NIGHTS 8 AT UNIVERSAL STUDIOS! 👻,['UNIVERSAL HALLOWEEN HORROR NIGHTS 8! 👻\n\nHa...,2018-10-31,7xkdm3c4Ks8,8766,271,3,18,halloween horror nights 8 universal studios,'universal halloween horror nights 8 happy hal...,halloween horror nights 8 universal studios 'u...


In [164]:
print(df_yt_ghib_sel.shape)
df_yt_ghib_sel.head(1)

(114, 11)


,video_title,video_caption,date_time,video_slug,video_views,video_likes,video_dislikes,video_comments,video_title_clean,video_caption_clean,content
4,Singapore Semi-Lockdown Diary Day 0 | Japanese...,"[""This thing called the 'circuit breaker' is h...",2020-04-07,632WZ2a-GCw,84167,1931,51,318,semilockdown diary day 0 japanese living,thing called 'circuit breaker' happening actua...,Singapore Semi-Lockdown Diary Day 0 | Japanese...


In [165]:
# let's create a channel col first before merging
df_yt_georgia_sel.loc[:, "channel"] = "GC"
df_yt_ghib_sel.loc[:, "channel"] = "GO"

C:\Users\yewto\anaconda3\lib\site-packages\pandas\core\indexing.py:845: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self.obj[key] = _infer_fill_value(value)
C:\Users\yewto\anaconda3\lib\site-packages\pandas\core\indexing.py:966: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self.obj[item] = s


In [166]:
# concatenate selected videos
df_yt_sel = pd.concat([df_yt_georgia_sel, df_yt_ghib_sel]).reset_index(drop=True)
df_yt_sel.shape

(248, 12)

In [167]:
# save out merged df
df_yt_sel.to_csv("../datasets/df_yt_sel.csv", index=False)

### YouTube Comments

#### Comments to Georgia Caney

In [168]:
df_yt_georgia_comments = pd.read_csv("../datasets/df_yt_georgia_comments_2020-10-17.csv")
df_yt_georgia_comments.shape

(11342, 6)

In [169]:
df_yt_georgia_comments.head(3)

,response_to,user,timestamp,comment,likes,replies_attracted
0,HALLOWEEN HORROR NIGHTS 8 AT UNIVERSAL STUDIOS! 👻,thattham,1 year ago,"I was a scare actor in killuminati, we hope u ...",4,CLAIM NOW!
1,HUGE FEBRUARY HAUL | MONKI & ROMWE,jess waldron,4 years ago,Loved this haul! Whenever you do one i always ...,NaN,View reply from Georgia Caney
2,HUGE FEBRUARY HAUL | MONKI & ROMWE,TheTravelCrush,4 years ago,OMGOSH!! Absolutely loved that kimono! I think...,NaN,NaN


In [170]:
# there are some duplicate scrapes
df_yt_georgia_comments.duplicated(keep="first").value_counts()

False    11316
True        26
dtype: int64

In [171]:
# drop duplicate scrapes
df_yt_georgia_comments = df_yt_georgia_comments.drop(df_yt_georgia_comments[df_yt_georgia_comments.duplicated(keep="first")].index)
df_yt_georgia_comments.shape

(11316, 6)

In [172]:
# let's clean the comments first
df_yt_georgia_comments["comment_clean"] = clean_text(df_yt_georgia_comments, "comment")
df_yt_georgia_comments.head(1)

,response_to,user,timestamp,comment,likes,replies_attracted,comment_clean
0,HALLOWEEN HORROR NIGHTS 8 AT UNIVERSAL STUDIOS! 👻,thattham,1 year ago,"I was a scare actor in killuminati, we hope u ...",4,CLAIM NOW!,scare actor killuminati hope u liked


In [173]:
df_yt_georgia_comments.info() # some missing values

<class 'pandas.core.frame.DataFrame'>
Int64Index: 11316 entries, 0 to 11341
Data columns (total 7 columns):
 #   Column             Non-Null Count  Dtype 
---  ------             --------------  ----- 
 0   response_to        11316 non-null  object
 1   user               11289 non-null  object
 2   timestamp          11316 non-null  object
 3   comment            11316 non-null  object
 4   likes              6443 non-null   object
 5   replies_attracted  5814 non-null   object
 6   comment_clean      11316 non-null  object
dtypes: object(7)
memory usage: 707.2+ KB


In [174]:
# find comments with 0 likes
mask = df_yt_georgia_comments["likes"] == 0
df_yt_georgia_comments[mask]

,response_to,user,timestamp,comment,likes,replies_attracted,comment_clean


In [175]:
# there are no comments with 0 likes
# comments with missing like values are most likely comments with 0 likes
# let's impute with 0
df_yt_georgia_comments["likes"].fillna("0", inplace=True)
df_yt_georgia_comments.info() # verify imputation

<class 'pandas.core.frame.DataFrame'>
Int64Index: 11316 entries, 0 to 11341
Data columns (total 7 columns):
 #   Column             Non-Null Count  Dtype 
---  ------             --------------  ----- 
 0   response_to        11316 non-null  object
 1   user               11289 non-null  object
 2   timestamp          11316 non-null  object
 3   comment            11316 non-null  object
 4   likes              11316 non-null  object
 5   replies_attracted  5814 non-null   object
 6   comment_clean      11316 non-null  object
dtypes: object(7)
memory usage: 707.2+ KB


In [176]:
df_yt_georgia_comments["likes"].str.contains("K").value_counts()

False    11315
True         1
Name: likes, dtype: int64

In [177]:
# noticed that some likes in the thousands are suffixed with a "K"
# remove the decimal point, and replace "K" with two zeroes
thousands = df_yt_georgia_comments["likes"].str.contains("K")
df_yt_georgia_comments.loc[thousands, "likes"] = df_yt_georgia_comments.loc[thousands, "likes"].str.replace(".", "").str.replace("K", "00")

In [178]:
df_yt_georgia_comments["likes"].str.contains("[a-zA-Z]").value_counts()

False    11266
True        50
Name: likes, dtype: int64

In [179]:
# and also that there are some alphabetical characters from scraping erroneous imputation
# this error arose from imputation when video has disabled comments and likes
# as such let's drop all such rows
error = df_yt_georgia_comments["likes"].str.contains("[a-zA-Z\.\s]")
df_yt_georgia_comments.drop(df_yt_georgia_comments[error].index, inplace=True)

In [180]:
# convert no. of likes to int
df_yt_georgia_comments["likes"] = df_yt_georgia_comments["likes"].astype(int)
df_yt_georgia_comments.info() # verify conversion

<class 'pandas.core.frame.DataFrame'>
Int64Index: 11256 entries, 0 to 11341
Data columns (total 7 columns):
 #   Column             Non-Null Count  Dtype 
---  ------             --------------  ----- 
 0   response_to        11256 non-null  object
 1   user               11229 non-null  object
 2   timestamp          11256 non-null  object
 3   comment            11256 non-null  object
 4   likes              11256 non-null  int32 
 5   replies_attracted  5754 non-null   object
 6   comment_clean      11256 non-null  object
dtypes: int32(1), object(6)
memory usage: 659.5+ KB


In [181]:
df_yt_georgia_comments["timestamp"].unique()

array(['1 year ago', '4 years ago', '10 months ago',
       '10 months ago (edited)', '5 months ago', '7 months ago',
       '2 years ago', '2 years ago (edited)', '1 week ago', '5 years ago',
       '3 years ago', '11 months ago', '2 months ago',
       '2 months ago (edited)', '5 months ago (edited)', '4 months ago',
       '4 months ago (edited)', '6 months ago', '1 month ago',
       '3 years ago (edited)', '6 months ago (edited)', '3 months ago',
       '1 year ago (edited)', '9 months ago', '8 months ago',
       '8 months ago (edited)', '7 months ago (edited)',
       '3 months ago (edited)', '1 day ago (edited)', '2 weeks ago',
       '3 weeks ago', '2 days ago', '6 years ago',
       '11 months ago (edited)', '4 weeks ago', '4 weeks ago (edited)',
       '1 month ago (edited)', '5 days ago', '9 months ago (edited)',
       '1 day ago', '32 minutes ago', '1 week ago (edited)',
       '2 weeks ago (edited)', '20 hours ago', '6 days ago',
       '3 weeks ago (edited)', '3 days ag

In [182]:
# parsing the timestamps requires inference
# or we could assume comments to have the same timestamps as videos

#### Match Comments to Videos

In [183]:
df_yt_georgia_sel.head(1)

,video_title,video_caption,date_time,video_slug,video_views,video_likes,video_dislikes,video_comments,video_title_clean,video_caption_clean,content,channel
0,HALLOWEEN HORROR NIGHTS 8 AT UNIVERSAL STUDIOS! 👻,['UNIVERSAL HALLOWEEN HORROR NIGHTS 8! 👻\n\nHa...,2018-10-31,7xkdm3c4Ks8,8766,271,3,18,halloween horror nights 8 universal studios,'universal halloween horror nights 8 happy hal...,halloween horror nights 8 universal studios 'u...,GC


In [184]:
df_yt_georgia_comments.head(1)

,response_to,user,timestamp,comment,likes,replies_attracted,comment_clean
0,HALLOWEEN HORROR NIGHTS 8 AT UNIVERSAL STUDIOS! 👻,thattham,1 year ago,"I was a scare actor in killuminati, we hope u ...",4,CLAIM NOW!,scare actor killuminati hope u liked


In [185]:
# match comments to selected video
videos = df_yt_georgia_sel["video_title"].values
videos = df_yt_georgia_comments["response_to"].str.contains("|".join(videos))
df_yt_georgia_comments_sel = df_yt_georgia_comments[videos] # create new df for selected comments
df_yt_georgia_comments_sel.shape

C:\Users\yewto\anaconda3\lib\site-packages\pandas\core\strings.py:1954: UserWarning: This pattern has match groups. To actually get the groups, use str.extract.
  return func(self, *args, **kwargs)


(6018, 7)

#### Comments to Ghib Ojisan

In [186]:
df_yt_ghib_comments = pd.read_csv("../datasets/df_yt_ghib_comments_2020-10-18.csv")
df_yt_ghib_comments.shape

(14518, 6)

In [187]:
df_yt_ghib_comments.head(3)

,response_to,user,timestamp,comment,likes,replies_attracted
0,【これが現実】エリート駐在員にストレスを吐き出してもらった結果…【シンガポール】,佐東忠司,1 year ago,始業時間に厳しく、終了時間がいい加減...\n時間に一番ルーズなのは日本人では？w\n友達が...,45,View reply
1,【これが現実】エリート駐在員にストレスを吐き出してもらった結果…【シンガポール】,no arara,1 year ago,日本の労働効率の悪さは、犯罪レベル。,23,NaN
2,【これが現実】エリート駐在員にストレスを吐き出してもらった結果…【シンガポール】,新たあらた,1 year ago,嫌なら辞めれば、というコメントは筋違い。海外赴任は期間限定だから数年で日本の元の環境に戻る。...,11,View reply


In [188]:
# there are some duplicate scrapes
df_yt_ghib_comments.duplicated(keep="first").value_counts()

False    14513
True         5
dtype: int64

In [189]:
# drop duplicate scrapes
df_yt_ghib_comments = df_yt_ghib_comments.drop(df_yt_ghib_comments[df_yt_ghib_comments.duplicated(keep="first")].index)
df_yt_ghib_comments.shape

(14513, 6)

In [190]:
# let's clean the comments first
df_yt_ghib_comments["comment_clean"] = clean_text(df_yt_ghib_comments, "comment")
df_yt_ghib_comments.head(1)

,response_to,user,timestamp,comment,likes,replies_attracted,comment_clean
0,【これが現実】エリート駐在員にストレスを吐き出してもらった結果…【シンガポール】,佐東忠司,1 year ago,始業時間に厳しく、終了時間がいい加減...\n時間に一番ルーズなのは日本人では？w\n友達が...,45,View reply,w


In [191]:
df_yt_ghib_comments.info() # some missing values

<class 'pandas.core.frame.DataFrame'>
Int64Index: 14513 entries, 0 to 14517
Data columns (total 7 columns):
 #   Column             Non-Null Count  Dtype 
---  ------             --------------  ----- 
 0   response_to        14513 non-null  object
 1   user               14488 non-null  object
 2   timestamp          14513 non-null  object
 3   comment            14513 non-null  object
 4   likes              12733 non-null  object
 5   replies_attracted  7433 non-null   object
 6   comment_clean      14513 non-null  object
dtypes: object(7)
memory usage: 907.1+ KB


In [192]:
# find comments with 0 likes
mask = df_yt_ghib_comments["likes"] == 0
df_yt_ghib_comments[mask]

,response_to,user,timestamp,comment,likes,replies_attracted,comment_clean


In [193]:
# there are no comments with 0 likes
# comments with missing like values are most likely comments with 0 likes
# let's impute with 0
df_yt_ghib_comments["likes"].fillna("0", inplace=True)
df_yt_ghib_comments.info() # verify imputation

<class 'pandas.core.frame.DataFrame'>
Int64Index: 14513 entries, 0 to 14517
Data columns (total 7 columns):
 #   Column             Non-Null Count  Dtype 
---  ------             --------------  ----- 
 0   response_to        14513 non-null  object
 1   user               14488 non-null  object
 2   timestamp          14513 non-null  object
 3   comment            14513 non-null  object
 4   likes              14513 non-null  object
 5   replies_attracted  7433 non-null   object
 6   comment_clean      14513 non-null  object
dtypes: object(7)
memory usage: 907.1+ KB


In [194]:
df_yt_ghib_comments["likes"].str.contains("K").value_counts()

False    14438
True        75
Name: likes, dtype: int64

In [195]:
# noticed that some likes in the thousands are suffixed with a "K"
# remove the decimal point, and replace "K" with two zeroes
thousands = df_yt_ghib_comments["likes"].str.contains("K")
df_yt_ghib_comments.loc[thousands, "likes"] = df_yt_ghib_comments.loc[thousands, "likes"].str.replace(".", "").str.replace("K", "00")

In [196]:
df_yt_ghib_comments["likes"].str.contains("[a-zA-Z]").value_counts()

False    14503
True        10
Name: likes, dtype: int64

In [197]:
# and also that there are some alphabetical characters from scraping erroneous imputation
# this error arose from imputation when video has disabled comments and likes
# as such let's drop all such rows
error = df_yt_ghib_comments["likes"].str.contains("[a-zA-Z\.\s]")
df_yt_ghib_comments.drop(df_yt_ghib_comments[error].index, inplace=True)

In [198]:
# convert no. of likes to int
df_yt_ghib_comments["likes"] = df_yt_ghib_comments["likes"].astype(int)
df_yt_ghib_comments.info() # verify conversion

<class 'pandas.core.frame.DataFrame'>
Int64Index: 14501 entries, 0 to 14517
Data columns (total 7 columns):
 #   Column             Non-Null Count  Dtype 
---  ------             --------------  ----- 
 0   response_to        14501 non-null  object
 1   user               14476 non-null  object
 2   timestamp          14501 non-null  object
 3   comment            14501 non-null  object
 4   likes              14501 non-null  int32 
 5   replies_attracted  7421 non-null   object
 6   comment_clean      14501 non-null  object
dtypes: int32(1), object(6)
memory usage: 849.7+ KB


In [199]:
# let's also match the comments to selected video titles
df_yt_ghib_comments.loc[:, "response_to"] = [s.replace("’", "'").replace("🇸🇬", "") for s in df_yt_ghib_comments["response_to"]]

In [200]:
# filter out all posts with japanese characters
only_eng = [title.isascii() for title in df_yt_ghib_comments["response_to"]]
df_yt_ghib_comments_eng = df_yt_ghib_comments[only_eng] # rewrite df
df_yt_ghib_comments_eng.shape

(6829, 7)

In [201]:
df_yt_ghib_comments_eng["timestamp"].unique()

array(['6 months ago', '5 months ago', '6 months ago (edited)',
       '1 year ago', '6 hours ago', '1 year ago (edited)', '9 months ago',
       '9 months ago (edited)', '8 months ago', '3 months ago',
       '7 months ago', '7 months ago (edited)', '3 months ago (edited)',
       '2 months ago', '2 years ago', '1 day ago', '10 months ago',
       '1 month ago', '8 months ago (edited)', '4 months ago',
       '4 weeks ago', '3 weeks ago', '3 weeks ago (edited)',
       '4 weeks ago (edited)', '2 weeks ago (edited)', '2 weeks ago',
       '2 hours ago', '2 days ago', '2 months ago (edited)',
       '5 months ago (edited)', '11 months ago', '1 week ago',
       '10 months ago (edited)', '4 months ago (edited)',
       '1 month ago (edited)', '11 months ago (edited)', '6 days ago',
       '2 years ago (edited)', '20 hours ago', '2 days ago (edited)',
       '4 days ago', '1 week ago (edited)', '12 hours ago', '9 hours ago',
       '3 days ago', '6 days ago (edited)', '11 hours ago',
    

In [202]:
# parsing the timestamps requires inference
# or we could assume comments to have the same timestamps as videos

#### Match Comments to Videos

In [203]:
df_yt_ghib_sel.head(1)

,video_title,video_caption,date_time,video_slug,video_views,video_likes,video_dislikes,video_comments,video_title_clean,video_caption_clean,content,channel
4,Singapore Semi-Lockdown Diary Day 0 | Japanese...,"[""This thing called the 'circuit breaker' is h...",2020-04-07,632WZ2a-GCw,84167,1931,51,318,semilockdown diary day 0 japanese living,thing called 'circuit breaker' happening actua...,Singapore Semi-Lockdown Diary Day 0 | Japanese...,GO


In [204]:
df_yt_ghib_comments_eng.head(1)

,response_to,user,timestamp,comment,likes,replies_attracted,comment_clean
106,Singapore Semi-Lockdown Diary Day 0 | Japanese...,NaN,6 months ago,Even in Singapore now,1,View 4 replies,even


In [205]:
# match comments to selected video
videos = df_yt_ghib_sel["video_title"].values
videos = df_yt_ghib_comments_eng["response_to"].str.contains("|".join(videos))
df_yt_ghib_comments_sel = df_yt_ghib_comments_eng[videos] # create new df for selected comments
df_yt_ghib_comments_sel.shape

C:\Users\yewto\anaconda3\lib\site-packages\pandas\core\strings.py:1954: UserWarning: This pattern has match groups. To actually get the groups, use str.extract.
  return func(self, *args, **kwargs)


(4572, 7)

#### Merge Selected Comments

In [206]:
df_yt_georgia_comments_sel.head(1)

,response_to,user,timestamp,comment,likes,replies_attracted,comment_clean
0,HALLOWEEN HORROR NIGHTS 8 AT UNIVERSAL STUDIOS! 👻,thattham,1 year ago,"I was a scare actor in killuminati, we hope u ...",4,CLAIM NOW!,scare actor killuminati hope u liked


In [207]:
df_yt_ghib_comments_sel.head(1)

,response_to,user,timestamp,comment,likes,replies_attracted,comment_clean
106,Singapore Semi-Lockdown Diary Day 0 | Japanese...,NaN,6 months ago,Even in Singapore now,1,View 4 replies,even


In [208]:
# before merging, let's add a channel col
df_yt_georgia_comments_sel["channel"] = "GC"
df_yt_ghib_comments_sel["channel"] = "GO"

<ipython-input-208-b36c558bb5c1>:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_yt_georgia_comments_sel["channel"] = "GC"
<ipython-input-208-b36c558bb5c1>:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_yt_ghib_comments_sel["channel"] = "GO"


In [209]:
# create new df for merged comments
df_yt_comments_sel = pd.concat([df_yt_georgia_comments_sel, df_yt_ghib_comments_sel], axis=0)
df_yt_comments_sel.shape

(10590, 8)

In [210]:
# save out merged df
df_yt_comments_sel.to_csv("../datasets/df_yt_comments_sel.csv", index=False)